## Scraping

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
URL = 'https://www.op.gg/leaderboards/tier'

In [3]:
response = requests.get(URL)

In [4]:
response.text[:50]

'<!DOCTYPE html><html lang="en" class="mode--deskto'

In [5]:
soup = BeautifulSoup(response.text)

In [ ]:
sorted(soup.__dir__())

In [ ]:
# <script id="__NEXT_DATA__" type="application/json">

In [6]:
import json

In [7]:
import pyperclip

In [8]:
data = json.loads(soup.find('script', {'id': "__NEXT_DATA__"}).string)

In [9]:
pyperclip.copy(data)

# paste to https://jsoneditoronline.org/

In [10]:
data = data['props']['pageProps']

In [65]:
data

{'region': 'eune',
 'tier': 'all',
 'type': 'ladder',
 'championById': {'1': {'id': 1,
   'key': 'Annie',
   'name': 'Annie',
   'image_url': 'https://opgg-static.akamaized.net/meta/images/lol/latest/champion/Annie.png',
   'evolve': [],
   'title': 'the Dark Child',
   'partype': 'Mana',
   'passive': {'name': 'Pyromania',
    'description': "After casting 4 spells, Annie's next offensive spell will stun the target.<br><br>Annie begins the game and respawns with Pyromania available.",
    'image_url': 'https://opgg-static.akamaized.net/meta/images/lol/latest/passive/Annie_Passive.png',
    'video_url': 'https://d28xe8vt774jo5.cloudfront.net/champion-abilities/0001/ability_0001_P1.webm'},
   'spells': [{'key': 'Q',
     'name': 'Disintegrate',
     'description': 'Annie hurls a Mana infused fireball, dealing damage and refunding the Mana cost if it destroys the target.',
     'max_rank': 5,
     'range_burn': [625],
     'cooldown_burn': [4],
     'cooldown_burn_float': [4],
     'cost

In [11]:
champions_mapping = {meta['id']: meta['name'] for meta in data['championById'].values()}

In [66]:
leaderboard = []
for meta in data['data']:
    for champion in meta['summoner']['most_champions']['champion_stats']:
        leaderboard.append({
            'rank': meta['rank'],
            'lp': next(mode['tier_info']['lp'] 
                       for mode in meta['summoner']['league_stats'] 
                       if mode['game_type'] == 'SOLORANKED' and mode['tier_info']['tier'] == 'CHALLENGER'
                      ),
            'champion': champions_mapping.get(champion['id']),
            'games': champion['play'],
            'winratio': round((champion['win'] / champion['play']) * 100),
        })
leaderboard[:5]

[{'rank': 1, 'lp': 1740, 'champion': 'Syndra', 'games': 86, 'winratio': 69},
 {'rank': 1,
  'lp': 1740,
  'champion': 'Cassiopeia',
  'games': 30,
  'winratio': 63},
 {'rank': 1, 'lp': 1740, 'champion': 'Xerath', 'games': 21, 'winratio': 62},
 {'rank': 2, 'lp': 1689, 'champion': 'Riven', 'games': 146, 'winratio': 68},
 {'rank': 2, 'lp': 1689, 'champion': 'Camille', 'games': 26, 'winratio': 69}]

In [13]:
import pandas as pd

In [67]:
df = pd.DataFrame(leaderboard)

In [68]:
len(df)

300

In [29]:
df.head()

,rank,lp,champion,games,winratio
0,1,1740,Syndra,86,69
1,1,1740,Cassiopeia,30,63
2,1,1740,Xerath,21,62
3,2,1689,Riven,146,68
4,2,1689,Camille,26,69


## Analysis

In [17]:
from sqlalchemy import Table, MetaData, create_engine

In [18]:
engine = create_engine('sqlite:///eune.db')

In [30]:
df.to_sql('eune_leaderboard', con=engine, if_exists='replace', index=False)

300

In [31]:
pd.read_sql_query("SELECT * FROM eune_leaderboard LIMIT 5", con=engine)

,rank,lp,champion,games,winratio
0,1,1740,Syndra,86,69
1,1,1740,Cassiopeia,30,63
2,1,1740,Xerath,21,62
3,2,1689,Riven,146,68
4,2,1689,Camille,26,69


In [21]:
def sql(query: str) -> pd.DataFrame:
    return pd.read_sql_query(query, con=engine)

In [22]:
def drop_table(table_name: str) -> None:
    Table(table_name, MetaData(), autoload_with=engine).drop(engine)

In [32]:
sql(
    """    
    SELECT 
        *,
        COUNT(rank) OVER (PARTITION BY champion) AS players_cnt
    FROM eune_leaderboard el
    """
).to_sql('eune_leaderboard', con=engine, if_exists='replace', index=False)

300

In [33]:
sql(
    """
    SELECT *
    FROM eune_leaderboard
    LIMIT 1
    """
)

,rank,lp,champion,games,winratio,players_cnt
0,15,1187,Aatrox,18,61,3


### The most played champion in challanger

In [64]:
sql(
    """
    WITH ranked_players_cnt AS (
        SELECT DISTINCT 
            champion, 
            players_cnt,
            DENSE_RANK() OVER (ORDER BY players_cnt DESC) AS rank
        FROM eune_leaderboard
    )
    SELECT champion, players_cnt
    FROM ranked_players_cnt
    WHERE rank <= 5
    """
)

,champion,players_cnt
0,Viego,9
1,Ashe,8
2,Sylas,8
3,Yone,8
4,Caitlyn,7
5,Draven,7
6,Ezreal,7
7,Graves,7
8,Hwei,7
9,Jhin,7


### The best mains on EUNE

In [50]:
sql(
    """
    SELECT DISTINCT rank, champion, games, winratio
    FROM eune_leaderboard
    WHERE winratio > 70
        AND games > 50
    ORDER BY games * winratio DESC
    """
)

,rank,champion,games,winratio
0,14,Kha'Zix,107,75
1,29,Olaf,113,71
2,51,Lulu,63,79
3,97,Gwen,64,77
4,13,Volibear,61,79
5,12,Kayle,53,72
